# TME 6 : Projet Detection de motifs


## Recheche de pattern (motifs) en utilisant les algoritmes randomisés

Les algorithmes randomisés prendre des décisions aléatoire plutôt que déterministes.
l'algorithme s'exécute différemment à chaque fois. Ils sont couramment utilisés dans des situations où aucun algorithme exact et rapide est connu. Nous allons implémenter deux algorithmes randomisés pour la recherche de motifs : Greedy Profile Motif Search et random Projections.


#### K Murali Sharane et Legendre--Despas Jeanne

1\. Nous allons réutiliser les fonctions du TME5 pour générer `t` séquences artificielles de taille `n`, et implanter dans chaque séquence un motif de taille `k` à des positions aléatoires avec `v` substitutions choisies aléatoirement.

In [21]:
import random
import numpy as np

nuc = ('A', 'C', 'G', 'T')

k=5 #taille de motif
v=1 #nb de positions variable dans le motif
t=3 #nb de sequences
n=10 #longuer des sequence

In [22]:
def generateRandomSequence(n, upper=True):
    """
    Génère une séquence nucléotidique aléatoire 
    entrée n : longueur de la sequence
    entrée upper : bool, si True les nucléotides seront en majuscule, False minuscule
    sortie sequence : une séquence nucléotidique aléatoire 
    """
    sequence = ""
    
    for i in range(n):
        if (upper) : 
            sequence += random.choice(nuc).upper()
        else : 
            sequence += random.choice(nuc).lower()
    
    return sequence

def modifierMotif(motif, nbpos,  upper=True):
    """
    Modifie nbpos positions d'un motif aléatoirement 
    entrée motif : motif à modifier
    entrée nbpos : nombre de positions
    entrée upper : bool, si True les nucléotides modifiés seront majuscule, False minuscule
    sortie motifM : motif modifié
    """
    
    #Cas erreur : bpos trop important par rapport à la taille du motif
    if (nbpos > len(motif)) : 
        return;
    
    motifM = list(motif)
    
    #On génère une liste de positions différentes aléatoirement
    positions = []
    for i in range(nbpos) : 
        pos = random.choice(range(len(motif)))
        while (pos in positions) :
            pos = random.choice(range(len(motif)))
        positions.append(pos)
    
    #On modifie les positions
    for pos in positions :
        nvNuc = random.choice(nuc)
        while (nvNuc.upper() == motifM[pos].upper()) :
            nvNuc = random.choice(nuc)
        
        if (upper) :
            motifM[pos] = nvNuc.upper()
        else : 
            motifM[pos] = nvNuc.lower()
        
    
    return "".join(motifM)



def implantMotifVar(k, v, t, n):
    """
    Génère des séquences aléatoires et les implante des motifs variables (un motif par séquence)
    entrée k : taille du motif
    entrée v : nombre de variations
    entrée t : nombre de séquences 
    entrée n : longueur des séquences
    sortie DNA : matrice de dimension txn avec les motifs implantés
    REMARQUE : La taille totale des séquences plus motif doit être égale à n, pensez à générer de séquence aléatoire de taille n-k pour pouvoir implanter un motif de taille k
    """

    #Génération des t séquences aléatoires de taille n-k
    sequences = [generateRandomSequence(n-k) for i in range(t)]
    
    #Génération des differents motifs avec une variation à chaque motif
    motif = generateRandomSequence(k, False) #Le premier motif est le motif original
    motifsModif = [modifierMotif(motif,v) for i in range(t-1)] #Liste des t-1 motifs avec v variations
    
    #Implantation des motifs à des positions aléatoires
    for i,seq in enumerate(sequences):
        
        #On choisi une position au hasard
        pos = random.choice(range( len(seq) + 1 ))
        if (i==0):
            sequences[i] = seq[:pos] + motif + seq[pos:]
        else : 
            sequences[i] = seq[:pos] + motifsModif[i-1] + seq[pos:]
            
    return sequences

adn = implantMotifVar(k, v, t, n)
print("Les",t,"séquence(s) de longueur",n,"implantée(s) avec avec un motif de longueur",k,"comportant",v,"variation(s) :\n")
for i,seq in enumerate(adn) :
    print("Séquence",i+1,":",seq)

Les 3 séquence(s) de longueur 10 implantée(s) avec avec un motif de longueur 5 comportant 1 variation(s) :

Séquence 1 : tgattAGGCT
Séquence 2 : ACCGtgaAtG
Séquence 3 : CAAtTattTT


2\. Faire une fonction pour sélectionner des positions de départ aléatoirement `s = (s1, …,st)`.


In [23]:
#creating vector s
#Get t random positions from 0 to n - k
def generateRandomS(sequences, k):
    """
    Génère un vecteur de position aléatoires
    entrée sequences: matrice de dimension txn contenant les sequences
    entrée k: taille du motif
    sortie s: vecteur de position aléatoires, une position par séquence
    REMARQUE les positions doivent être inférieur à n-k
    """
    return [random.randint(0,len(seq)-k) for seq in sequences]
    

s = generateRandomS(adn, k)
print("Génération d'un vecteur de positions aléatoire pour les motif de taille",k,"dans la matrice de séquences : ")
for i,seq in enumerate(adn) :
    print("Séquence",i+1,":",seq)
print("Le vecteur de positions aléatoires :",s)

Génération d'un vecteur de positions aléatoire pour les motif de taille 5 dans la matrice de séquences : 
Séquence 1 : tgattAGGCT
Séquence 2 : ACCGtgaAtG
Séquence 3 : CAAtTattTT
Le vecteur de positions aléatoires : [2, 3, 5]


3\. Extraire les motifs en utilisant le vecteur `s`, et construire un profil (matrice de fréquence). Attention, utiliser les pseudocount 1 pour éviter les plus tard les probabilités à zéro.


In [24]:
def extractSeqs(s, seqs, k):
    """
    Extraire les motifs des séquences à l'aide de positions s
    entrée s: vecteur contenant les positions de départs
    entrée seqs: matrice de dimension txn contenant les séquences
    entrée k: taille du motif
    sortie motifs: liste de motifs de taille k extrait des séquences
    """
    #Liste des sequences extraites à l'aide de la positions de départ correpondante dans s
    return [seq[s[i]:s[i]+k] for i,seq in enumerate(seqs)]
        

#Construire matrice de fréquence
def profile(motifs, k, nuc):
    """
    Construire une matrice de fréquence
    entrée motifs: liste de motifs
    entrée k: taille du motif
    entrée nuc: alphabet
    sortie matriceFreq: le matrice de fréquence
    """
    q = len(nuc) #Hauteur de la matrice
    #Matruce de hauteur q (une ligne pour chaque nucléotide) et de longueur k (une colonne pour chaque position du motif)
    matriceFreq = np.zeros((q, k))
    
    #On parcourt tous les motifs
    for motif in motifs:
        #On parcourt tous les nucléotides du motif
        for i,n in enumerate(motif):
            for j in range(q) : #On test quel nucléotide à la position i
                if (n == nuc[j].upper() or n == nuc[j].lower()) :
                    matriceFreq[j][i] += 1 #On incrémente le compteur de nucléotide pour la position i
                    break
                    
    return matriceFreq + 1 #On utilise le pseudo-count 1

print("La matrice de séquence :")
for i,seq in enumerate(adn) :
    print("   Séquence",i+1,":",seq)
    
print("\nLe vecteur de positions aléatoires :",s)

motifs = extractSeqs(s, adn, k) #On extrait les motifs de taille k au positions données
print("\nLes motifs extraits des positions du vecteur de positions aléatoires:",motifs)

MF = profile(motifs, k, nuc) #On construit la matrice de fréquence

print("\nMatrice de fréquence (ACGT) construite à partir des motifs extraits\n",MF)


La matrice de séquence :
   Séquence 1 : tgattAGGCT
   Séquence 2 : ACCGtgaAtG
   Séquence 3 : CAAtTattTT

Le vecteur de positions aléatoires : [2, 3, 5]

Les motifs extraits des positions du vecteur de positions aléatoires: ['attAG', 'GtgaA', 'attTT']

Matrice de fréquence (ACGT) construite à partir des motifs extraits
 [[3. 1. 1. 3. 2.]
 [1. 1. 1. 1. 1.]
 [2. 1. 2. 1. 2.]
 [1. 4. 3. 2. 2.]]


4\. Transformer la matrice de fréquence en PWM. Il faut diviser chaque élément par la somme de ses colonnes. Faire aussi une fonction pour calculer la sequence consensus d'une matrice de frequence.

In [25]:
motifs = [motif.upper() for motif in motifs] #On s'assure que tous les motifs soit en majuscule

def generatePWM(MF):
    """
    Transforme la matrice de fréquence en PWM
    entrée MF: matrice de fréquence
    sortie PWM: matrice de probabilité (poids positions)
    """
    #On divise chaque case par la somme des colonnes
    return MF/np.sum(MF, axis = 0)[0]

PWM = generatePWM(MF)
print("La matrice de fréquence :")
print(MF)
print("\nLa matrice de poids positions associée à notre matrice de fréquence")
print(PWM)

La matrice de fréquence :
[[3. 1. 1. 3. 2.]
 [1. 1. 1. 1. 1.]
 [2. 1. 2. 1. 2.]
 [1. 4. 3. 2. 2.]]

La matrice de poids positions associée à notre matrice de fréquence
[[0.42857143 0.14285714 0.14285714 0.42857143 0.28571429]
 [0.14285714 0.14285714 0.14285714 0.14285714 0.14285714]
 [0.28571429 0.14285714 0.28571429 0.14285714 0.28571429]
 [0.14285714 0.57142857 0.42857143 0.28571429 0.28571429]]


In [26]:
def getConsensus(MF, k, nuc):
    """
    Renvoie la sequence consensus d'une matrice de fréquence
    entrée MF: matrice de fréquence
    entrée k: taille du motif
    entrée nuc: alphabet
    sortie seqCons: sequence consensus
    """
    seqCons = ""
    #On parcourt chaque colonne (position) de la matrice de fréquence
    for freq in np.transpose(MF):
        vMax = np.max(freq) #On récupère la fréquence max
        indice = np.where(freq==vMax)[0][0] #On récupère le premier idice de ligne (nucléotide) avec cette fréquence
        seqCons += nuc[indice] #On ajoute le nucléotides correspondant à la séquence consensus
   
    return seqCons

#On calcule la séquence consensus de notre matrice de fréquence
seqCons = getConsensus(MF, k, nuc)
print("Matrice de fréquence :")
print(MF)
print("\nLa séquence consensus de notre matrice de fréquence est", seqCons)

Matrice de fréquence :
[[3. 1. 1. 3. 2.]
 [1. 1. 1. 1. 1.]
 [2. 1. 2. 1. 2.]
 [1. 4. 3. 2. 2.]]

La séquence consensus de notre matrice de fréquence est ATTAA


5\. Faire une fonction pour calculer la probabilité d'un motif de taille `k` selon une PWM.


In [27]:
def probability(PWM, motif):
    """
    Calcul la probalité d'un motif selon PWM
    entrée PWM: matrice de probabilité (poids positions)
    entrée motif: motif
    sortie prob: probalité Prob(motif|PWM)
    """
    pos = 0; prob = 1
    #print (PWM)
    for n in motif: #On parcourt le motif
        i = nuc.index(n)
        #print (i, pos)
        prob *=PWM[i, pos] #On calcule la probabilité de chacune des positions du motifs
        pos +=1
    return prob

prob = probability(PWM, seqCons)
print("La probabilité de la séquence consensus", seqCons, "est", prob)

La probabilité de la séquence consensus ATTAA est 0.012851787945498894


6\. Faire une fonction pour calculer le pMostProbkmer d'une séquence, voir un exemple dans les slides de cours.

In [28]:
import re
#Repère les kmer peu complexe pour éliminer le bruit de fond
def isComplexe(kmer, reLow):
    """
    entrée kmer : kmer à évaluer
    entrée reLow : niveau de répétition à éliminer  
    sortie : true si le kmer est peu complexe, false sinon
    """
    k = str(len(kmer))
    reLow_fois_même_lettre = '([ACTG]*A){'+ str(reLow) +'}|([ACTG]*C){'+ str(reLow) +'}|([ACTG]*T){'+ str(reLow) +'}|([ACTG]*G){'+ str(reLow) +'}'
    deux_lettres_repetee = '([ACTG]*AC){3}|([ATCG]*AT){3}|([ATCG]*AG){3}|([ATCG]*TA){3}|([ATCG]*TG){3}|([ATCG]*TC){3}|([ATCG]*CG){3}|([ATCG]*CA){3}|([ATCG]*CT){3}|([ATCG]*GA){3}|([ATCG]*GT){3}|([ATCG]*GC){3}'
    #uniquement_deux_lettres = '[AC]{'+k+'}|[AT]{'+k+'}|[AG]{'+k+'}|[TG]{'+k+'}|[TC]{'+k+'}|[GC]{'+k+'}'
    
    p = re.compile(reLow_fois_même_lettre + deux_lettres_repetee)
    return not p.match(kmer)



In [29]:

def pMostProbkmer(PWM, k, sequence, reLow):
    """
    calcul la position du k-mer de haute complexité le plus probable dans la séquence
    entrée PWM: matrice de probabilité (poids positions)
    entrée k: taille du motif
    entrée sequence: séquence nucleotidique
    entrée relow : niveau de répétition dans les kmer à éliminer
    sortie s: la position du k-mer le plus probable dans la sequence
    """
    
    #Initialisation
    
    s = 0 # position
    kmer = sequence[s:s+k]
    
    #On cherche le premier complexe de haute complexité
    while not isComplexe(kmer, reLow) : 
        s += 1
        kmer = sequence[s:s+k]
        if s >= len(sequence)-k : 
            break
            
    #On calcule la probabilité du kmer en fonction de la matrice de fréquence
    proba = probability(PWM, sequence[s:s+k])
    
    i = s + 1
    #On cherche le kmer complexe avec la meilleure probabilité
    for pos in range(i, len(sequence)-k+1): #On parcourt toutes les positions

        kmer = sequence[pos:pos+k]
        if (isComplexe(kmer, reLow) and probability(PWM, kmer) > proba):
        #if (probability(PWM, kmer) > proba):
            s = pos
            proba = probability(PWM, kmer)
    
    return s

#Test avec notre sequence concensus
seqTest = "GCC" + seqCons + "GGATACT"
print("Séquence :", seqTest)
posSeqCons = pMostProbkmer(PWM, k, seqTest, 5)
print ("La position du kmer le plus probable par raport à notre matrice de fréquence est :", posSeqCons)

Séquence : GCCATTAAGGATACT
La position du kmer le plus probable par raport à notre matrice de fréquence est : 3


<font color = blue>
    On retrouve bien notre séquence consensus à la position 3.
</font>

7\. Faire une fonction pour obtenir les nouvelles positions de départ `s = (s1, …,st)`, c’était à dire les positions qui contiennent les k-mer le plus probables.

In [30]:
def getNewS(PWM, k, sequences, relow):
    """
    Trouve les nouvelles positions des k-mer le plus probables
    entrée PWM: matrice de probabilité (poids positions)
    entrée k: taille du motif
    entrée sequence: séquence nucleotidique
    entrée reLow: seuil de répétitions des lettres de kmer
    sortie s: vecteur avec les positions les plus problables
    """
    return [pMostProbkmer(PWM, k, sequence.upper(), relow) for sequence in sequences]

#On calcule la position du kmer le plus probable au niveau de chaque séquence
newS = getNewS(PWM, k, adn, 5)

print("Les sequences :")
for i,seq in enumerate(adn) :
    print("  Séquence",i+1,":",seq)
    
print("Les premières positions alétoires :", s)
print("La séquence consensus issue de ces positions :",seqCons)
print("Les nouvelles positions des kmer les plus probables selon la première matrice de fréquence :",newS)
print("Les kmer associés :",extractSeqs(newS, adn, k))

Les sequences :
  Séquence 1 : tgattAGGCT
  Séquence 2 : ACCGtgaAtG
  Séquence 3 : CAAtTattTT
Les premières positions alétoires : [2, 3, 5]
La séquence consensus issue de ces positions : ATTAA
Les nouvelles positions des kmer les plus probables selon la première matrice de fréquence : [2, 3, 2]
Les kmer associés : ['attAG', 'GtgaA', 'AtTat']


8\. La condition d’arrêt de l'algorithme est le non changement de la matrice de fréquence d'une itération à l'autre. Faire une fonction pour comparer deux matrices et détecter le changement. 
Faire aussi une fonction pour obtenir le score d'une matrice de fréquence : la somme de max de chaque colonne

In [31]:
def changeProfile(P1, P2):
    """
    Compare deux matrice
    entrée P1: matrice de fréqueince
    entrée P2: matrice de fréquence
    sortie: True si les matrices sont différents, False au contraire
    """
    return (P1 != P2).any()


def getScore(MF, k):
    """
    Renvoie le score de MF, la somme des max de chaque colonne
    entrée P: matrice de fréquence
    entrée k: taille du motif
    sortie sc: score
    """
    return np.sum(np.max(MF, axis = 0))

#Anciennew positions
print("Les premières positions aléatoires :", s)
print("La matrice de poids positions associée:")
print(MF)
sc = getScore(MF, k)
print("Le score de la matrice de poids position est de",sc)

#Nouvellew positions
newS = getNewS(PWM, k, adn, 5)
print("\nLes positions des k-mer les plus probables selon la matrice de poids positions :", newS)
newMF = profile(extractSeqs(newS, adn, k), k, nuc)
print("La nouvelle matrice de poids positions :")
print(newMF)
if (changeProfile(MF, newMF)) : 
    print("La nouvelle matrice de fréquence est différente.")
else :
    print("La nouvelle matrice de fréquence est identique à la précédente")
newsc = getScore(MF, k)
print("Le score de la nouvelle matrice de poids position est",sc)


Les premières positions aléatoires : [2, 3, 5]
La matrice de poids positions associée:
[[3. 1. 1. 3. 2.]
 [1. 1. 1. 1. 1.]
 [2. 1. 2. 1. 2.]
 [1. 4. 3. 2. 2.]]
Le score de la matrice de poids position est de 15.0

Les positions des k-mer les plus probables selon la matrice de poids positions : [2, 3, 2]
La nouvelle matrice de poids positions :
[[3. 1. 1. 4. 2.]
 [1. 1. 1. 1. 1.]
 [2. 1. 2. 1. 2.]
 [1. 4. 3. 1. 2.]]
La nouvelle matrice de fréquence est différente.
Le score de la nouvelle matrice de poids position est 15.0


9\. Implementer l'algorithme ``GreedyProfileMotifSearch`` en utilisant les fonctions precedentes. 


In [32]:
def GreedyProfileMotifSearch(sequences, t, n, k, reLow):
    """
    Implémente l'algorithme GreedyProfileMotifSearch 
    entrée sequences: matrice de dimension txn contenant les séquences 
    entrée t : nombre de séquences 
    entrée n : longueur des séquences 
    entrée k : taille du motif 
    sortie s : vecteur de positions de départ ayant le meilleur motif 
    sortie bestScore : le score associé à s
    """
    
    #Génère un premier n-upplet de positions aléatoire
    s = generateRandomS(sequences, k)
    
    bestScore = 0
    
    #Créer un profil Pr à partir des séquences des positions de départ
    Pr = profile(extractSeqs(s, sequences, k), k, nuc)
    PWM = generatePWM(Pr)
    
    while (getScore(Pr, k) > bestScore) :
        
        bestScore = getScore(Pr, k)
        
        #Touver de P-most-probable-k-mer pour chaque sequence
        s = [pMostProbkmer(PWM, k, sequence.upper(), reLow) for sequence in sequences]
        
        #Nouveau profil
        Pr = profile(extractSeqs(s, sequences, k), k, nuc)
        PMW = generatePWM(Pr)
    
    return s, bestScore


s, bestScore = GreedyProfileMotifSearch(adn, t, n, k, 5)

print("Les sequences implantées : ")
for i,seq in enumerate(adn) :
    print("  Séquence",i+1,":",seq)

print("\nLes meilleurs positions trouvées sont", s, "associées à un score de", bestScore)
print("\nLes motifs correspondants sont", extractSeqs(s, adn, k))
        

Les sequences implantées : 
  Séquence 1 : tgattAGGCT
  Séquence 2 : ACCGtgaAtG
  Séquence 3 : CAAtTattTT

Les meilleurs positions trouvées sont [0, 3, 3] associées à un score de 15.0

Les motifs correspondants sont ['tgatt', 'GtgaA', 'tTatt']


10\. Avez vous trouvez le motif implanté? Réexécuter l’algorithme plusieurs fois pour le trouver. 

<font color = blue>
Oui, nous retrouvons le motif implanté.
</font>

11\. Vous avez certainement observer que l’algorithme ne produire pas toujours la même sortie et que les résultats dépendent de la sélection aléatoire des positions de départ `s`. Pour augmenter nous chances de retrouvez les bons motifs, nous allons implémenter une version itérative ``GreedyProfileMotifSearchIte`` 
qui a chaque fois sauvegarde `s` et le score du profile associé à `s`, après `I` itération, l’algorithme renvoie le vecteur `s` ayant le plus grand score.

In [33]:
def GreedyProfileMotifSearchIte(sequences, t, n, k, It, reLow):
    """
    Implémente l'algorithme GreedyProfileMotifSearch iteratif
    entrée séquences: matrice de dimension txn contenant les séquences 
    entrée t : nombre de séquences 
    entrée n : longueur des séquences 
    entrée k: taille du motif 
    entrée It: nombre d'iterations
    sortie positions: dictionnaire clé=score, valeur= vecteur s 
    sortie consensus: dictionnaire clé=score, valeur= consensus sequence du motif
    """
    #Dictionnaire avec les scores associés aux positions des séquences
    pos = {}
    #Dictionnaire avec les scores associés à la séquence consensus des motifs les plus probables
    consensus = {}
    
    for i in range(It):
        s, bestScore = GreedyProfileMotifSearch(sequences, t, n, k, reLow)
        seqCons = getConsensus(profile(extractSeqs(s, sequences, k), k, nuc), k, nuc)
        #print(extractSeqs(s, sequences, k))
        if (isComplexe(seqCons, reLow)) :
            pos[bestScore] = s
            consensus[bestScore] = seqCons
    
    return pos, consensus


pos, consensus = GreedyProfileMotifSearchIte(adn, t, n, k, 50, 5)

print("Les sequences implantées : ")
for i,seq in enumerate(adn) :
    print("  Séquence",i+1,":",seq)
    
print("\nLes positions des séquences :")
print(pos)
print("\nLes séquences consensus :")
print(consensus)


Les sequences implantées : 
  Séquence 1 : tgattAGGCT
  Séquence 2 : ACCGtgaAtG
  Séquence 3 : CAAtTattTT

Les positions des séquences :
{16.0: [2, 3, 2], 18.0: [0, 4, 3], 13.0: [5, 0, 0], 14.0: [0, 0, 0], 17.0: [0, 4, 4], 15.0: [5, 3, 2], 11.0: [4, 1, 2]}

Les séquences consensus :
{16.0: 'ATTAA', 18.0: 'TGATT', 13.0: 'AAACT', 14.0: 'AAATT', 17.0: 'TGATT', 15.0: 'ATGAT', 11.0: 'AAGAC'}


12\. Tester algorithme  ``GreedyProfileMotifSearchIte`` sur vos données de chipSeq. N'oubliez pas de chercher les motifs dans le brin complémentaire et faire un merge de résultats.

In [34]:
k=7; reLow=5
n = 100
def readFasta(genome, n):
    sequence = []
    file = open(genome, "r")
    sequence = []
    for s in file:
        if s[0] != ">":
            sequence.append(s.strip().upper())
    sequenceStr = "".join(sequence)
    sequence = [sequenceStr[i:i+n] for i in range(0, len(sequenceStr), n)]
    sequenceRet = [x for x in sequence if x]
    return sequenceRet

genome = "../Sequence_by_Peaks_1.fasta" #Remplacer par votre fichier

sequencesFasta = readFasta(genome, n)
t = len(sequencesFasta)

In [35]:
def reversecompl(seq):
    """Renvoie le brin complémentaire d’une séquence.
    entrée : sequence de nucléotides (brin sens)
    sortie : sequence de nucléotides (brin complementaire)
    >>> reversecompl('AACGTGGCA')
    'TGCCACGTT'
    """
    compl = {'A': 'T', 'C': 'G', 'G': 'C', 'T':'A', 'a': 't', 'c': 'g', 'g': 'c', 't':'a'}
    nvSeq = ""
    for i in range(len(seq)-1,-1,-1) :
        nvSeq = nvSeq + compl[seq[i]]
    
    return nvSeq

In [36]:
def mergeMotifs(consensus, consensusRevComp):
    ''' 
    Merge motifs trouvés dans les brins sens et complementaire
    entrée motifs : un dictionnaire contenant les motifs du brin sens et leurs distances
    entrée motifsRevComp : un dictionnaire contenant les motifs du brin complementaire et leurs distances
    sortie allMotif: merge de motifs et motifsRevComp
    '''
    return {score: consensus for dico in (consensus, consensusRevComp) for score, consensus in dico.items()}

<font color = blue>
Nous testons d'abord avec le premier algorithme de tri : on supprime les motifs avec 5 occurences du même nucléotides ou plus et les motifs comportant 3 doublets.
</font>

In [38]:
t = len(sequencesFasta)
n = len(sequencesFasta[0])
print(reLow)

#RECHERCHE DES MOTIFS DANS LE BRIN SENS
print("Recherche des motifs dans le brin sens...\n")
posFastaSens, consensusFastaSens = GreedyProfileMotifSearchIte(sequencesFasta, t, n, k, 100, reLow)

#RECHERCHE DES MOTIFS DANS LE REVERSE COMPLÉMENT
print("\nRecherche des motifs dans le brin reverse complément...\n")
sequencesFastaCompl = [reversecompl(sequence) for sequence in sequencesFasta]
posFastaCompl, consensusFastaCompl = GreedyProfileMotifSearchIte(sequencesFastaCompl, t, n, k, 100, reLow)

#RECHERCHE DANS LES DEUX BRINS
print("\nRecherche des motifs dans les deux brins...\n")
consensusFasta = mergeMotifs(consensusFastaSens, consensusFastaCompl)
print("\nRecherche des motifs terminée\n")

5
Recherche des motifs dans le brin sens...


Recherche des motifs dans le brin reverse complément...


Recherche des motifs dans les deux brins...


Recherche des motifs terminée



Afficher les top motifs (leurs sequence consensus) de meilleurs scores.

In [39]:
def printTopFMotifsScore(consensus, top):

    motifsSort = sorted(consensus.keys())
    motifsSort.reverse()
    for i in range(min(top, len(consensus))):
        print(motifsSort[i], consensus[motifsSort[i]])

print("Meilleures sequences consensus sens")
printTopFMotifsScore(consensusFastaSens, 3)
print("\nMeilleures seéquences consensus reverse complément")
printTopFMotifsScore(consensusFastaCompl, 3)
print("\nMeilleures séquences consensus")
printTopFMotifsScore(consensusFasta, 3)


Meilleures sequences consensus sens
1356.0 AAATATT
1348.0 AATTATA
1343.0 ATATTTA

Meilleures seéquences consensus reverse complément
1380.0 TTAATTA
1362.0 TAATTAT
1341.0 ATAATTA

Meilleures séquences consensus
1380.0 TTAATTA
1362.0 TAATTAT
1356.0 AAATATT


<font color = blue>
    On peut lire beaucoup de bruit de fond avec des motifs comportant uniquement deux lettres répétées. On test un nouvel algorithme de tri qui va supprimer les motifs qui ne contiennent uniquement deux lettres. 
</font>

In [40]:
def isComplexe(kmer, reLow):
    """
    entrée kmer : kmer à évaluer
    entrée reLow : niveau de répétition à éliminer  
    sortie : true si le kmer est peu complexe, false sinon
    """
    k = str(len(kmer))
    reLow_fois_même_lettre = '([ACTG]*A){'+ str(reLow) +'}|([ACTG]*C){'+ str(reLow) +'}|([ACTG]*T){'+ str(reLow) +'}|([ACTG]*G){'+ str(reLow) +'}|'
    deux_lettres_repetee = '([ACTG]*AC){3}|([ATCG]*AT){3}|([ATCG]*AG){3}|([ATCG]*TA){3}|([ATCG]*TG){3}|([ATCG]*TC){3}|([ATCG]*CG){3}|([ATCG]*CA){3}|([ATCG]*CT){3}|([ATCG]*GA){3}|([ATCG]*GT){3}|([ATCG]*GC){3}|'
    uniquement_deux_lettres = '[AC]{'+k+'}|[AT]{'+k+'}|[AG]{'+k+'}|[TG]{'+k+'}|[TC]{'+k+'}|[GC]{'+k+'}'
        
    p = re.compile(reLow_fois_même_lettre + deux_lettres_repetee + uniquement_deux_lettres)

    return not p.match(kmer)

In [41]:
reLow = 5
#RECHERCHE DES MOTIFS DANS LE BRIN SENS
print("Recherche des motifs dans le brin sens...\n")
posFastaSens, consensusFastaSens = GreedyProfileMotifSearchIte(sequencesFasta, t, n, k, 100, reLow)


#RECHERCHE DES MOTIFS DANS LE REVERSE COMPLÉMENT
print("\nRecherche des motifs dans le brin reverse complément...\n")
sequencesFastaCompl = [reversecompl(sequence) for sequence in sequencesFasta]
posFastaCompl, consensusFastaCompl = GreedyProfileMotifSearchIte(sequencesFastaCompl, t, n, k, 100, reLow)


#RECHERCHE DANS LES DEUX BRINS
print("\nRecherche des motifs dans les deux brins...\n")
consensusFasta = mergeMotifs(consensusFastaSens, consensusFastaCompl)

print("\nRecherche des motifs terminée\n")

Recherche des motifs dans le brin sens...


Recherche des motifs dans le brin reverse complément...


Recherche des motifs dans les deux brins...


Recherche des motifs terminée



In [42]:
print("Meilleures sequences consensus sens")
printTopFMotifsScore(consensusFastaSens, 3)
print("\nMeilleures seéquences consensus reverse complément")
printTopFMotifsScore(consensusFastaCompl, 3)
print("\nMeilleures séquences consensus")
printTopFMotifsScore(consensusFasta, 3)


Meilleures sequences consensus sens
1287.0 TTCAATA
1258.0 TGAAATT
1250.0 ATCTAAT

Meilleures seéquences consensus reverse complément
1273.0 TTATGAT
1231.0 TTATGAA
1217.0 AATTACA

Meilleures séquences consensus
1287.0 TTCAATA
1273.0 TTATGAT
1258.0 TGAAATT


<font color = blue>
Nous n'avons pas réussi à retrouver notre motif.
</font>

13\. Nous allons implémenter l'algorithme ``randomProjection``. D'abord, faites la fonction `getRandomFixePositions` pour générer une projection de l à k, voir un exemple dans les slides de cours. Faire aussi la fonction `generateKey` qui extrait les caractères du motif puis génère une cle qui représente la projection.

In [43]:
def getRandomFixePositions(k, n):
    """
    Genere une projection de k vers n
    entrée k: nombre de positions du motif original
    entrée n: nombre de positions choisi pour la projection 
    sortie projection: liste de positions choisi aléatoirement
    """
    if (n > k) :
        return;
    
    #La liste des position choisies pour la projection
    projection = []
    
    #On choisi n position
    for i in range(n):
        pos = random.randint(0,k-1)
        while (pos in projection) : #Verfication que la position n'a pas déjà été choisie
            pos = random.randint(0,k-1)
        projection.append(pos)
    projection.sort()
    return projection

lR = getRandomFixePositions(7, 4)
print("Positions aléatoires :",lR)

def generateKey(positions, motif):
    """
    extrait les caractères du motif et génère la cle de la projection
    entrée positions : liste de positions qui represent la projection
    entrée motif : motif de taille k
    sortie cle : cle de la projection
    """
    return "".join([list(motif)[pos] for pos in positions])

print("Clef générée à partie de 01234567 :",generateKey(lR, "01234567"))


Positions aléatoires : [0, 2, 3, 5]
Clef générée à partie de 01234567 : 0235


14\. Implémenter l'algorithme ``randomProjection``.

In [44]:
def randomProjection(k, n, sequences, relow):
    """
    Implémente l'algorithme randomProjection
    entrée k : nombre de positions du motif
    entrée n : nombre de positions de la projection 
    entrée sequences : matrice de dimension txn contenant les séquences 
    sortie motifs : dictionaire, cle = projection, valeur= frequence
    sortie motifsSeq:  dictionaire, cle = projection, valeur= original motif
    """
    motifs  = {}; motifsSeq  = {}
    
    
    #On selectionne k positions aléatoirement parmis les l nucléotides du motif
    fixPos = getRandomFixePositions(k, n)
    
    dna = "".join(sequences)
    #Pour chaque k-tuple dans les sequences, on trouve son bucket de taille n
    for pos in range(len(dna)-k+1) :
        motif = dna[pos:pos+k]
        if isComplexe(motif, reLow) : 
            #On calcule la projection du k-tuple avec nos positions fixes
            projection = generateKey(fixPos, motif)

            if projection not in motifs : #nv bucket 
                motifs[projection] = 1
                motifsSeq[projection] = [motif]
            else : #ajout au bucket
                motifs[projection] += 1
                motifsSeq[projection].append(motif)

    return motifs, motifsSeq



#adnTest = ['TTAACGGCAC', 'GCTCACGCCA', 'TACCGGCCGT']

k=5 #taille de motif
v=1 #nb de positions variable dans le motif
t=3 #nb de sequences
n=10 #longuer des sequences

adnTest = implantMotifVar(k, v, t, n)
print(adnTest)
motifsProj, motifsSeq = randomProjection(5, 3, adnTest, 5)

motifsProjSort = sorted(motifsProj.items(), key=lambda x: x[1], reverse=True)

for i in range(3):
    print("Bucket", i+1, ":", motifsProjSort[i][0], motifsSeq[motifsProjSort[i][0]])


['ATGagacaGA', 'AGCgacaGCC', 'CgacaCGGGT']
Bucket 1 : gaa ['agaca', 'Cgaca', 'Cgaca']
Bucket 2 : acG ['gacaG', 'gacaG']
Bucket 3 : Cgc ['GCgac', 'CCgac']


15\. Avez vous trouvez le motif implanté? Rexécuter l’algorithme plusieurs fois pour augmenter les chances de le trouver. 

<font color = blue>
    Nous trouvons le motif implanté dans le bucket le plus fréquent.
</font>

16\. Implémenter la version itérative de l’algorithme ``randomProjection``. 

In [45]:
def randomProjIt(sequences, k, v, nuc, It, reLow):
    """
    Implémente l'algorithme randomProjection version iteractive
    entrée sequences : matrice de dimension txn contenant les séquences 
    entrée k : nombre de positions du motif
    entrée v : nombre de positions de la projection 
    entrée nuc : alphabet
    entrée It: nombre d'iterations
    sortie score : meilleur score
    sortie motifs :  liste de motifs associés au meilleur score
    """
    
    motifs = []; scores = []
    
    for i in range(It):
        print(i)
        
        motifsProj, motifsSeq = randomProjection(k, v, sequences, reLow)
        

        #Tri des motifs selon la taille des buckets
        motifsProjSort = sorted(motifsProj.items(), key=lambda x: x[1], reverse=True)
     
        #ajout du meilleur motif
        motifs.append(motifsSeq[motifsProjSort[0][0]])
        scores.append(motifsProjSort[0][1])
    
    return max(scores), motifs[scores.index(max(scores))] 

score, seqsMotif = randomProjIt(adn, 7, 4, nuc, 100, 6)

print (score, seqsMotif)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
2 ['tgattAG', 'tTattTT']


17\. Tester l'algorithme  ``randomProjection`` sur vos données de chipSeq. N'oubliez pas de chercher les motifs dans le brin complémentaire et faire un merge de résultats. Puis générér le LOGO du motif trouvé.

<font color = blue>
    On test d'abord l'algorithme randomProjection avec deux positions variables
</font>

In [49]:
k = 8
v = 6 #nombre de positions fixes
it = 100
reLow = 6

score, seqsMotif = randomProjIt(sequencesFasta+sequencesFastaCompl, k, v, nuc, it, reLow)

print(score, seqsMotif)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
72 ['AAGCAACA', 'CAGCAATA', 'CAGCAATA', 'TAGCAACA', 'CAGCAAAA', 'TAGCAACA', 'CAGCAATA', 'CAGCAAAA', 'CAGCAATA', 'CAGCAAAA', 'AAGCAACA', 'CAGCAAAA', 'TAGCAAGA', 'CAGCAAAA', 'CAGCAAAA', 'GAGCAAGA', 'CAGCAAAA', 'CAGCAAAA', 'CAGCAAAA', 'TAGCAAAA', 'CAGCAAAA', 'AAGCAATA', 'CAGCAAAA', 'TAGCAAGA', 'TAGCAATA', 'CAGCAATA', 'GAGCAAAA', 'CAGCAAAA', 'AAGCAAGA', 'CAGCAATA', 'CAGCAATA', 'CAGCAATA', 'CAGCAATA', 'TAGCAACA', 'CAGCAATA', 'CAGCAAAA', 'CAGCAAAA', 'CAGCAAAA', 'CAGCAAAA', 'GAGCAAGA', 'AAGCAACA', 'CAGCAAAA', 'CAGCAAAA', 'CAGCAATA', 'TAGCAATA', 'TAGCAATA', 'TAGCAATA', 'TAGCAATA', 'CAGCAAAA', 'CAGCAAAA', 'TAGCAATA', 'AAGCAAGA', 'CAGCAAAA', 'CAGCAAAA', 'CAGCAAAA', 'CAGCAAAA', 'AAGCAAGA', 'CAGCAAAA', 'AAGCAATA'

In [47]:
dicoMotifOcc = {}

for mot in seqsMotif :
    if mot in dicoMotifOcc :
        dicoMotifOcc[mot] += 1
    else :
        dicoMotifOcc[mot] = 1
print("Le meilleur bucket trouvé est : ")
print(dicoMotifOcc)

Le meilleur bucket trouvé est : 
{'TATTGCTG': 15, 'TATGGCTG': 1, 'TTTTGCTG': 29, 'TTTAGCTG': 9, 'TTTGGCTG': 4, 'TATCGCTG': 7, 'TCTAGCTG': 2, 'TTTCGCTG': 1, 'TCTCGCTG': 1, 'TGTCGCTG': 1, 'TGTAGCTG': 2}


Motif LOGO:
<img src="motif_randomProj_deuxVar.png">

<font color = blue>
    Le meilleur bucket trouvé pour cette itération avec deux variations : {'CAGCAATA': 15, 'CAGCAAAA': 29, 'CAGCGAAA': 1, 'CAGCGACA': 1, 'CAGCGAGA': 1, 'CAGCTAAA': 8, 'CAGCTACA': 2, 'CAGCGATA': 8, 'CAGCCAAA': 4, 'CAGCCATA': 1, 'CAGCTAGA': 2}. On retrouve le motif CAGCAAAA comme séquence consensus de ce bucket.
</font>

<font color = blue>
Nous testons maintenat l'algorithme RandomProjection avec une seule variation
</font>

In [48]:
k = 8
v = 7 #nombre de positions fixes
it = 100
reLow = 6

score, seqsMotif = randomProjIt(sequencesFasta+sequencesFastaCompl, k, v, nuc, it, reLow)

print(score, seqsMotif)

dicoMotifOcc = {}

for mot in seqsMotif :
    if mot in dicoMotifOcc :
        dicoMotifOcc[mot] += 1
    else :
        dicoMotifOcc[mot] = 1
        
print(dicoMotifOcc)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
44 ['CAGCAATA', 'CAGCAATA', 'CAGCAAAA', 'CAGCAATA', 'CAGCAAAA', 'CAGCAATA', 'CAGCAAAA', 'CAGCAAAA', 'CAGCAAAA', 'CAGCAAAA', 'CAGCAAAA', 'CAGCAAAA', 'CAGCAAAA', 'CAGCAAAA', 'CAGCAAAA', 'CAGCAATA', 'CAGCAAAA', 'CAGCAATA', 'CAGCAATA', 'CAGCAATA', 'CAGCAATA', 'CAGCAATA', 'CAGCAAAA', 'CAGCAAAA', 'CAGCAAAA', 'CAGCAAAA', 'CAGCAAAA', 'CAGCAAAA', 'CAGCAATA', 'CAGCAAAA', 'CAGCAAAA', 'CAGCAAAA', 'CAGCAAAA', 'CAGCAAAA', 'CAGCAAAA', 'CAGCAAAA', 'CAGCAATA', 'CAGCAAAA', 'CAGCAATA', 'CAGCAAAA', 'CAGCAATA', 'CAGCAAAA', 'CAGCAAAA', 'CAGCAATA']
{'CAGCAATA': 15, 'CAGCAAAA': 29}


<font color = blue>
    Le meilleur bucket trouvé pour cette itération avec une variations : {'CAGCAATA': 15, 'CAGCAAAA': 29}. On retrouve le motif CAGCAAAA
</font>

Motif LOGO:
<img src="motif_randomProj_uneVar.png">

18\. Vous avez sans doute remarqué que les algos implémentés trouvent, suivant des motifs peu complexes, réviser les implémentations pour régler/diminuer ce problème.

<font color = blue>
    Nous avons fait une fonction isComplexe(kmer, reLow) qui retire les motifs peu complexe. Nous utilisons cette fonction dans pMostProbkmer(PWM, k, sequence, reLow) pour trouver directement les k-mer les plus probable de haute compléxité ainsi que dans GreedyProfileMotifSearchIte(sequences, t, n, k, It, reLow) pour trier les séquences consensus. Nous avons également utiliser cette fonction pour n'ajouter aux bucket que des motifs de haute compléxité dans la fonction randomProjection(k, n, sequences, relow).
</font>